In [1]:
import pandas as pd
import numpy as np
from collections import Counter,defaultdict
import networkx as nx

In [2]:
data = pd.read_csv('NAS_processed_032523.csv')

In [3]:
Exposure_Layer3 = data['Exposure_Layer3']
Outcome_Layer3 = data['Outcome_Layer3']

In [4]:
Exposure_Layer3_dict = defaultdict(dict)
Exposure_Layer3_exposure2node = dict()

for i,k in enumerate(Counter(Exposure_Layer3).keys()):
    Exposure_Layer3_dict[i]['count']=Counter(Exposure_Layer3)[k]
    Exposure_Layer3_dict[i]['property']=k
    Exposure_Layer3_exposure2node[k]=i

In [5]:
Outcome_Layer3_dict = defaultdict(dict)
Outcome_Layer3_outcome2node = dict()
existing_node = len(Exposure_Layer3_exposure2node)
for i,k in enumerate(Counter(Outcome_Layer3).keys()):
    if k not in Counter(Exposure_Layer3).keys():
        node = existing_node+1
        existing_node = node
        Outcome_Layer3_dict[node]['count']=Counter(Outcome_Layer3)[k]
        Outcome_Layer3_dict[node]['property']=k
        Outcome_Layer3_outcome2node[k] = node

In [6]:
layer3_2nodes = {**Outcome_Layer3_outcome2node,**Exposure_Layer3_exposure2node}

In [7]:
layer3_nodes2name = dict()

for k in layer3_2nodes.keys():
    layer3_nodes2name[layer3_2nodes[k]]=k

In [8]:
#transfer to graph tuple
Exposure_Layer3_nodes = [(k,Exposure_Layer3_dict[k]) for k in Exposure_Layer3_dict.keys()]
Outcome_Layer3_nodes = [(k,Outcome_Layer3_dict[k]) for k in Outcome_Layer3_dict.keys()]

In [9]:
adj_list = set()

for i,v in enumerate(Exposure_Layer3):
    e_node = layer3_2nodes[v]
    o_node = layer3_2nodes[Outcome_Layer3[i]]
    if (e_node,o_node) not in adj_list:
        adj_list.add((e_node,o_node))
#how many edges
#adj_list
len(adj_list)

394

In [10]:
node_list = defaultdict(list)

for a,b in list(adj_list):
    node_list[a].append(b)

#path of where each node can go
#node_list

## hypothesis generation

In [11]:
mediate_pool = []
visited = set()
for i in range(190): #number of node-190
    if i not in visited:
        tovisit = [(i,str(i),[i])]
        visited.add(i)
        while tovisit:
            n,p,temp = tovisit.pop(0)
            if n not in node_list.keys():
                #print(p)
                mediate_pool.append(p)
            else:
                for nn in node_list[n]:
                    if nn not in temp:
                        newtemp=temp+[nn]
                        visited.add(nn)
                        tovisit.append((nn,p+'->'+str(nn),newtemp))

In [12]:
mediation_name_pool = []
for st in mediate_pool:
    li = st.split('->')
    if len(li)>=3:
        #print(st)#print mediation pool node numbers
        fulname = ''
        for vv in li:
            fulname+='->'+layer3_nodes2name[int(vv)]
        mediation_name_pool.append(fulname)

In [13]:
mediation_name_pool_ = [v[2:] for v in mediation_name_pool]

for v in mediation_name_pool_:
    print(v)

short-term PM2.5 mass->genes-450K->FEV1
short-term PM2.5 mass->genes-450K->FEF25%-75%
short-term PM2.5 mass->genes-450K->FVC
short-term PM2.5 elemental components->genes-450K->FEV1
short-term PM2.5 elemental components->genes-450K->FEF25%-75%
short-term PM2.5 elemental components->genes-450K->FVC
short-term black carbon->genes-selected->diastolic blood pressure
short-term black carbon->genes-selected->mortality
short-term black carbon->genes-selected->FEV1
short-term black carbon->genes-selected->mitochondrial DNA 8-OHdG lesions
short-term black carbon->genes-selected->FEV1/FVC
short-term black carbon->genes-selected->FVC
short-term black carbon->genes-selected->systolic blood pressure
short-term black carbon->genes-selected->incident cancer
short-term black carbon->genes-450K->FEV1
short-term black carbon->genes-450K->FEF25%-75%
short-term black carbon->genes-450K->FVC
short-term black carbon->genes-selected->mitochondrial DNA copy number->FEV1
short-term black carbon->genes-selected-

In [14]:
Layer3_mediation = pd.DataFrame(data = mediation_name_pool_,columns=['path'])
Layer3_mediation.to_csv('Layer3_mediationpool.csv')

## network centrality analysis

In [15]:
G = nx.DiGraph()

G.add_nodes_from(Outcome_Layer3_nodes)
G.add_nodes_from(Exposure_Layer3_nodes)
G.add_edges_from(list(adj_list))

In [16]:
from collections import defaultdict

data = []
for k in dict(G.in_degree()).keys():
    data.append((layer3_nodes2name[k],G.in_degree()[k],G.out_degree()[k],nx.in_degree_centrality(G)[k],nx.out_degree_centrality(G)[k],nx.betweenness_centrality(G)[k],nx.closeness_centrality(G)[k]))

In [17]:
df = pd.DataFrame(data, columns =['Node name', 'In degree', 'Out degree','In degree centrality','Out degree centrality','Betweeness centrality','Closeness Centranilty'])

In [18]:
df

,Node name,In degree,Out degree,In degree centrality,Out degree centrality,Betweeness centrality,Closeness Centranilty
0,total white blood cell,3,0,0.016216,0.000000,0.000000,0.016216
1,neutrophil concentration,3,0,0.016216,0.000000,0.000000,0.016216
2,monocytes concentration,3,0,0.016216,0.000000,0.000000,0.016216
3,lymphocyte concentration,3,0,0.016216,0.000000,0.000000,0.016216
4,eosinophil concentration,3,0,0.016216,0.000000,0.000000,0.016216
...,...,...,...,...,...,...,...
181,toenail cadmium,2,2,0.010811,0.010811,0.000118,0.010811
182,toenail mercury,0,2,0.000000,0.010811,0.000000,0.000000
183,stressful life events,0,1,0.000000,0.005405,0.000000,0.000000
184,self-regulation,0,1,0.000000,0.005405,0.000000,0.000000


In [19]:
df.to_csv('Layer3_networkstatistics.csv',sep='\t')  

In [20]:
#top nodes sorted by in degree centrality
INDEGREE=df.sort_values(by=['In degree'],ascending=False)
INDEGREE.head(7)
#INDEGREE.to_csv('Layer3_INDEGREE.csv',sep='\t')  

,Node name,In degree,Out degree,In degree centrality,Out degree centrality,Betweeness centrality,Closeness Centranilty
132,genes-selected,30,10,0.162162,0.054054,0.009540,0.162331
15,QT interval,25,0,0.135135,0.000000,0.000000,0.137756
28,inflammation biomarkers,17,0,0.091892,0.000000,0.000000,0.095351
109,aging clocks,17,3,0.091892,0.016216,0.001403,0.116393
14,untargeted metabolomics profiling,15,0,0.081081,0.000000,0.000000,0.081399
6,diastolic blood pressure,14,0,0.075676,0.000000,0.000000,0.136912
7,systolic blood pressure,14,0,0.075676,0.000000,0.000000,0.136912


In [21]:
#top nodes sorted by in betweeness centrality
BETWEENNESS=df.sort_values(by=['Betweeness centrality'],ascending=False)
BETWEENNESS.head(7)
#BETWEENNESS.to_csv('Layer3_BETWEENNESS.csv',sep='\t')  

,Node name,In degree,Out degree,In degree centrality,Out degree centrality,Betweeness centrality,Closeness Centranilty
132,genes-selected,30,10,0.162162,0.054054,0.009540,0.162331
109,aging clocks,17,3,0.091892,0.016216,0.001403,0.116393
106,mitochondrial DNA copy number,6,5,0.032432,0.027027,0.001344,0.093816
103,patella lead,3,23,0.016216,0.124324,0.001111,0.016216
104,tibia lead,3,23,0.016216,0.124324,0.001111,0.016216
127,genes-450K,9,3,0.048649,0.016216,0.001021,0.053736
110,leukocyte telomere length,4,4,0.021622,0.021622,0.000588,0.098830


In [22]:
#top nodes sorted by in outdegree centrality
OUTDEGREE=df.sort_values(by=['Out degree'],ascending=False)
OUTDEGREE.head(7)
#OUTDEGREE.to_csv('Layer3_OUTDEGREE.csv',sep='\t')  

,Node name,In degree,Out degree,In degree centrality,Out degree centrality,Betweeness centrality,Closeness Centranilty
104,tibia lead,3,23,0.016216,0.124324,0.001111,0.016216
103,patella lead,3,23,0.016216,0.124324,0.001111,0.016216
72,short-term PM2.5 mass,1,18,0.005405,0.097297,0.000455,0.005405
107,blood lead,2,15,0.010811,0.081081,0.000392,0.010811
75,intermediate-term PM2.5 mass,0,14,0.000000,0.075676,0.000000,0.000000
74,short-term black carbon,0,14,0.000000,0.075676,0.000000,0.000000
117,long-term black carbon,0,11,0.000000,0.059459,0.000000,0.000000


## Top 2 communities

In [23]:
from networkx.algorithms import community

In [24]:
c = community.greedy_modularity_communities(G)
#c

In [25]:
for v in c[0]:
    print(layer3_nodes2name[v])

untargeted metabolomics profiling
QT interval
FEV1/FVC
FEV1
MMEF
short-term ozone
short-term temperature
FVC
intermediate-term ozone
intermediate-term temperature
long-term PM2.5 elemental components
long-term PM2.5 mass
mitochondrial DNA 8-OHdG lesions
incident cancer
short-term nitrate
intermediate-term nitrate
long-term organic carbon
long-term nitrate
incident metabolic syndrome
metabolic syndrome components
long-term temperature
long-term NO2
long-term ozone
long-term sulfate
long-term temperature-outdoor
one-carbon metabolites
mitochondrial DNA copy number
aging clocks
leukocyte telomere length
urinary arsenic
urinary cadmium
urinary manganese
urinary mercury
antihypertensive medication
genes-selected
short-term relative humidity
long-term ammonium
dietary flavonoids
anxiety
hostility
happiness
life satisfaction
vitamin E intake
serum tocopherol levels
short-term temperature standardad deviation
intermediate-term temperature standardad deviation
intermediate-term relative humidit

In [26]:
for v in c[1]:
    print(layer3_nodes2name[v])

short-term PM2.5 mass
glomerular filtration rate
short-term black carbon
intermediate-term PM2.5 mass
intermediate-term PM2.5 elemental components
short-term PM2.5 elemental components
intermediate-term black carbon
hemoglobin concentration
short-term NO2
leukocyte distribution
intermediate-term NO2
FEF25%-75%
inflammation biomarkers
short-term organic carbon
intermediate-term organic carbon
short-term sulfate
intermediate-term sulfate
fasting blood glucose
heart rate variability markers
microRNA in extracellular vesicles
perceived stress
microRNA in blood leukocytes
augmentation index
augmentation pressure
particle radioactivity
short-term particle number
short-term CO
short-term SO2
genes-450K
manganese intake
road proximity
oxidative DNA damage
intermediate-term particle number
short-term NO
intermediate-term CO
intermediate-term NO
serum uric acid
incident chronic kidney disease
blood urea nitrogen
